In [ ]:
#basic stuff :
!pip install langchain langchain-openai langchain-community openai python-dotenv


INFO: pip is looking at multiple versions of langchain-openai to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of langchain-community to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.0/76.0 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 40.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 1.2 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32

In [ ]:
import os
from dotenv import load_dotenv
from google.colab import userdata
from langchain.chat_models import ChatOpenAI


load_dotenv()
NEBIUS_KEY = userdata.get("NEBIUS_API_KEY")
OPENAI_KEY = userdata.get("OPENAI_API_KEY")
NEBIUS_BASE = "https://api.studio.nebius.com/v1"

llm = ChatOpenAI(
    model="meta-llama/Meta-Llama-3.1-8B-Instruct",
    temperature=0.6,
    max_tokens=512,
    openai_api_base=NEBIUS_BASE,
    openai_api_key=NEBIUS_KEY
)


/tmp/ipython-input-2800840027.py:12: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(


##react design pattern

In [ ]:
!pip install wikipedia

  Preparing metadata (setup.py) ... done
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11678 sha256=73f5dc5cd0263a903f93712432da4e24b1394286f20fc55f28f39eee0af221de
  Stored in directory: /root/.cache/pip/wheels/63/47/7c/a9688349aa74d228ce0a9023229c6c0ac52ca2a40fe87679b8
Successfully built wikipedia


In [ ]:
from langchain import hub
prompt = hub.pull("hwchase17/react")

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}

In [ ]:
from langchain.agents import create_react_agent, AgentExecutor, load_tools

# Load tools
tools = load_tools(["wikipedia"])

agent = create_react_agent(llm, tools, prompt =prompt)


agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

agent_executor.invoke({"input": "who is hozier"})




> Entering new AgentExecutor chain...
Thought: The question is asking for information about a person.
Action: wikipedia
Action Input: HozierPage: Hozier
Summary: Andrew John Hozier-Byrne (born 17 March 1990), known professionally as Hozier ( HOH-zee-ər), is an Irish musician. His music primarily draws from folk, soul and blues, often using religious and literary themes and taking political or social justice stances.
His debut single, "Take Me to Church" (2013), became a rock radio hit in the U.S., peaked at number two on the Billboard Hot 100 and was certified multi-platinum in several countries. His eponymous debut studio album (2014) has been certified 6× platinum in Ireland and multi-platinum in several other countries. His EP Nina Cried Power (2018), which featured the title track as a single, reached number one on the Billboard Adult Alternative Songs chart. His second album, Wasteland, Baby! (2019), debuted atop the Irish Albums Chart and the Billboard 200, and was certified go

{'input': 'who is hozier', 'output': 'Hozier is an Irish musician.'}

##Fake multi-agent

In [ ]:
from langchain.agents import initialize_agent, AgentType, load_tools

# Create specialized agents
research_tools = load_tools(["wikipedia"], llm=llm)
math_tools = load_tools(["llm-math"], llm=llm)

#for simplicity, lets use initialzie_agent, however we'd get similar results on using react agent + executor

research_agent = initialize_agent(research_tools, llm, agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)
math_agent = initialize_agent(math_tools, llm, agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

def coordinator(query):
    """Decide who handles what."""
    if "calculate" in query or "*" in query:
        return math_agent.run(query)
    return research_agent.run(query)

print(coordinator("Who is the president of India?"))
print(coordinator("Calculate 345 * 23"))
